This notebook will have a fully functional and better conversational chatbot.

### Part 1: Load embedding model, llm and create a retriever

In [1]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
import os
from langchain_community.llms import LlamaCpp

In [2]:
# Function to download the embedding model
def download_embedding_model():
    """
    Function to download the embedding model
    
    Args:
    model_name : str : Name of the model to download
    
    Returns:
    embeddings : object : Embedding model object
    """
    
    embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
    return embeddings

In [3]:
embeddings = download_embedding_model()

C:\Users\vinee\Desktop\Challenges\Medical-Chatbot\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\vinee\Desktop\Challenges\Medical-Chatbot\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
load_dotenv()
PINECONE_API_KEY = "e485b6b8-b6f4-4e95-b3b8-081584058e80"
PINECONE_API_ENV = "us-east-1"
PINECONE_INDEX_NAME = "medicine"

In [5]:
vectorStore = PineconeVectorStore(index_name=PINECONE_INDEX_NAME, embedding=embeddings)
retriever = vectorStore.as_retriever(search_kwargs={"k":3})

In [6]:
local_llm = "../models/BioMistral-7B.Q4_K_M.gguf"
llm = LlamaCpp(model_path= 
local_llm,temperature=0.3,max_tokens=4096,top_p=1,n_ctx= 4096, verbose=False)

## Create the retrieval Chain

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [8]:
prompt_search_query = ChatPromptTemplate.from_messages([
MessagesPlaceholder(variable_name="chat_history"),
("user","{input}"),
("user","Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

prompt_get_answer = ChatPromptTemplate.from_messages([
("system", "Answer the user's questions based in detail on the following context. If you need more information or clarification, ask for it:\\n\\n{context}"),
MessagesPlaceholder(variable_name="chat_history"),
("user","{input}"),
])

In [13]:
retriever_chain = create_history_aware_retriever(llm, retriever, prompt_search_query)

In [14]:
document_chain=create_stuff_documents_chain(llm,prompt_get_answer)

In [15]:
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [24]:
# Initialize chat history
chat_history = [SystemMessage(content="You are a knowledgeable medical chatbot that provides information on common medical conditions based on user symptoms. Offer potential diagnoses, preventive measures, and advice clearly and accurately.")]

# Function to handle a single turn of the conversation
def handle_conversation(input_text):
    global chat_history
    # Invoke the retrieval chain
    response = retrieval_chain.invoke({
        "chat_history": chat_history,
        "input": input_text
    })

    print(response)
    
    # Extract and print the response
    answer = response['answer']
    print("AI:", answer)
    
    # Update chat history with the user's input and AI's response
    chat_history.append(HumanMessage(content=input_text))
    chat_history.append(AIMessage(content=answer))

In [ ]:
# Main loop to continuously take user input and update chat history
print("Start chatting with the bot! Type 'exit' to end the conversation.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    handle_conversation(user_input)

Start chatting with the bot! Type 'exit' to end the conversation.


You:  What is diabetes mellitus and how to prevent it?


{'chat_history': [SystemMessage(content='You are a knowledgeable medical chatbot that provides information on common medical conditions based on user symptoms. Offer potential diagnoses, preventive measures, and advice clearly and accurately.')], 'input': 'What is diabetes mellitus and how to prevent it?', 'context': [Document(page_content='. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 421'), Document(page_content='::· .. :" "\'\'. ", .·: . ::·. ... 1 " £. :::· ��,��,��r��1 ����!��� .�-=\'.��:=;=··�=-\n�1·--t-i� :.. :�"\' ;.::,� ----\'-· -\'-\' .:.�...._-:-: ---:�..... ; . . . ........ \n1-- ,-.'), Document(page_content='25 ..... ,,. \'- -- I \'\\/• · \'\\, ....... �·, .... \'-.::..-7 --" ;�·/ "\\\\\\1,1•• 10, ... ;<� .. �:::-·-· ;;--·-Q// \'"\'\\ {\\\\V\\ I \n\'-. \'°\'.;.\'" ...... -�--_....I V \'.t�\\, I 3 ·-. \'\'-....:: • .i._-:.·-:.··············\'/ /,\\··-.,.\\\\\\\' \\ ·. - ...... _ ,. ,\'\\i\'�.')], 'answer': ''}
